In [3]:
import numpy as np
import pandas as pd

leggo il dataset e stampo un campione (prime 5 righe)

In [5]:
dataset = pd.read_csv('sms_dataset.csv', encoding="ISO-8859-1")
dataset.sample(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
4307,ham,Ha ha - had popped down to the loo when you he...,NaN,NaN,NaN
3866,ham,"Alright we're hooked up, where you guys at",NaN,NaN,NaN
2704,spam,FreeMsg: Fancy a flirt? Reply DATE now & join ...,NaN,NaN,NaN
1969,spam,You have won a guaranteed å£200 award or even ...,NaN,NaN,NaN
1615,ham,Mm i had my food da from out,NaN,NaN,NaN


numero righe e colonne del dataset

In [7]:
dataset.shape

(5572, 5)

## Data Cleaning

stampo le info del dataset per fare una panoramica

In [8]:
dataset.info

<bound method DataFrame.info of         v1                                                 v2 Unnamed: 2  \
0      ham  Go until jurong point, crazy.. Available only ...        NaN   
1      ham                      Ok lar... Joking wif u oni...        NaN   
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3      ham  U dun say so early hor... U c already then say...        NaN   
4      ham  Nah I don't think he goes to usf, he lives aro...        NaN   
...    ...                                                ...        ...   
5567  spam  This is the 2nd time we have tried 2 contact u...        NaN   
5568   ham              Will Ì_ b going to esplanade fr home?        NaN   
5569   ham  Pity, * was in mood for that. So...any other s...        NaN   
5570   ham  The guy did some bitching but I acted like i'd...        NaN   
5571   ham                         Rofl. Its true to its name        NaN   

     Unnamed: 3 Unnamed: 4  
0           NaN        NaN

si può notare che ci sono delle colonne inutili (ultime 3), quindi vanno eliminate

In [11]:
dataset.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)
dataset.sample(5)

,v1,v2
2473,ham,Ok lor wat time Ì_ finish?
5547,spam,Had your contract mobile 11 Mnths? Latest Moto...
2870,ham,I actually did for the first time in a while. ...
4740,ham,Many more happy returns of the day. I wish you...
3014,ham,&lt;#&gt; mins but i had to stop somewhere f...


rinomino le colonne v1 e v2 con nomi migliori

In [15]:
dataset.rename(columns={'v1':'target', 'v2':'text'}, inplace=True)
dataset.sample(5)

,target,text
14,ham,I HAVE A DATE ON SUNDAY WITH WILL!!
377,ham,Well there's not a lot of things happening in ...
1457,spam,CLAIRE here am havin borin time & am now alone...
4487,ham,Call me when you get the chance plz &lt;3
5320,ham,But we havent got da topic yet rite?


ora devo traformare il target "ham/spam" in codifica binaria 0/1 quindi ho bisogno di sickitLearn e più nello specifico di LabelEncoder che viene utilizzato per convertire etichette categoriche in numeri interi in modo che possano essere utilizzate come input per algoritmi di machine learning. 


In [14]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [16]:
dataset['target'] = encoder.fit_transform(dataset['target'])
dataset.sample(5)

,target,text
305,0,Yup... From what i remb... I think should be c...
4228,0,Have you bookedthe hut? And also your time off...
5426,0,Oh yeah! And my diet just flew out the window
3493,1,You are being contacted by our dating service ...
3143,0,"Haha I heard that, text me when you're around"


quindi ora gli spam saranno indicati come 1, gli ham come 0

controllo se sono presenti valori nulli o duplicati, e nel caso risolvo

In [18]:
dataset.isnull().sum()

target    0
text      0
dtype: int64

non ci sono valori nulli, procedo a verificare eventuali duplicati

In [19]:
dataset.duplicated().sum()

403

procedo a rimuovere i 403 duplicati

In [22]:
dataset = dataset.drop_duplicates(keep='first')

In [23]:
dataset.duplicated().sum()

0

ho risolto! Ora il dataset ha 0 nulli e 0 duplicati